In [1]:
%matplotlib widget
import matplotlib.pyplot as plt

import numpy as np
import spikeinterface.full as si

import pandas as pd
from pathlib import Path

In [2]:
si.__version__

'0.102.2'

In [3]:
import os
# Specify the source directory
source_dir = r'H:\Work\SpikeSorting\kiehnlab\Treadmill_clustered_txt'

# Ensure the directory exists
if not os.path.exists(source_dir):
    raise ValueError(f"Directory '{source_dir}' does not exist")

# Initialize an empty list to store paths
folder_paths = []
folder_paths  = [os.path.join(source_dir, f) for f in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, f))]


In [4]:
base_folder = Path(folder_paths[0])
base_folder 

WindowsPath('H:/Work/SpikeSorting/kiehnlab/Treadmill_clustered_txt/Rec_open-field_xx_xx_-210629_VGlut2-ChR2-1_-210629-085256')

In [6]:
filename = base_folder.name
recording_file_tev = base_folder / (filename+'.tev')
recording = si.read_tdt(recording_file_tev,stream_name='Wav1')
recording 

TdtRecordingExtractor: 1 channels - 24414.062500 Hz - 1 segments - 49,770,496 samples 
                       2,038.60s (33.98 minutes) - float32 dtype - 189.86 MiB

In [7]:
channel_ids = recording.get_channel_ids()
fs = recording.get_sampling_frequency()
num_chan = recording.get_num_channels()
num_segments = recording.get_num_segments()

print(f'Channel ids: {channel_ids}')
print(f'Sampling frequency: {fs}')
print(f'Number of channels: {num_chan}')
print(f"Number of segments: {num_segments}")

Channel ids: ['1']
Sampling frequency: 24414.0625
Number of channels: 1
Number of segments: 1


In [8]:
from probeinterface import generate_linear_probe
from probeinterface.plotting import plot_probe

probe = generate_linear_probe(num_elec=1, ypitch=20, contact_shapes='circle', contact_shape_params={'radius': 6})

# the probe has to be wired to the recording
probe.set_device_channel_indices(np.arange(1))

recording = recording.set_probe(probe)
# plot_probe(probe)

In [9]:
si.plot_traces(recording,time_range = (10,100),backend = 'ipywidgets')

AppLayout(children=(TimeSlider(children=(Dropdown(description='segment', options=(0,), value=0), Button(icon='…

In [15]:
si.get_default_sorter_params('tridesclous')

{'freq_min': 400.0,
 'freq_max': 5000.0,
 'detect_sign': -1,
 'detect_threshold': 5,
 'common_ref_removal': False,
 'nested_params': None,
 'n_jobs': 12,
 'chunk_duration': '1s',
 'progress_bar': True,
 'mp_context': None,
 'max_threads_per_process': 1}

In [16]:
si.get_default_sorter_params('mountainsort5')

{'scheme': '2',
 'detect_threshold': 5.5,
 'detect_sign': -1,
 'detect_time_radius_msec': 0.5,
 'snippet_T1': 20,
 'snippet_T2': 20,
 'npca_per_channel': 3,
 'npca_per_subdivision': 10,
 'snippet_mask_radius': 250,
 'scheme1_detect_channel_radius': 150,
 'scheme2_phase1_detect_channel_radius': 200,
 'scheme2_detect_channel_radius': 50,
 'scheme2_max_num_snippets_per_training_batch': 200,
 'scheme2_training_duration_sec': 300,
 'scheme2_training_recording_sampling_mode': 'uniform',
 'scheme3_block_duration_sec': 1800,
 'freq_min': 300,
 'freq_max': 6000,
 'filter': True,
 'whiten': True,
 'delete_temporary_recording': True}

In [17]:
si.get_default_sorter_params('spykingcircus2')

{'general': {'ms_before': 2, 'ms_after': 2, 'radius_um': 100},
 'waveforms': {'max_spikes_per_unit': 200,
  'overwrite': True,
  'sparse': True,
  'method': 'energy',
  'threshold': 0.25},
 'filtering': {'freq_min': 150, 'dtype': 'float32'},
 'detection': {'peak_sign': 'neg', 'detect_threshold': 4},
 'selection': {'method': 'smart_sampling_amplitudes',
  'n_peaks_per_channel': 5000,
  'min_n_peaks': 20000,
  'select_per_channel': False},
 'clustering': {'legacy': False},
 'matching': {'method': 'circus-omp-svd', 'method_kwargs': {}},
 'apply_preprocessing': True,
 'shared_memory': True,
 'cache_preprocessing': {'mode': 'memory',
  'memory_limit': 0.5,
  'delete_cache': True},
 'multi_units_only': False,
 'job_kwargs': {'n_jobs': 0.8},
 'debug': False}

In [17]:
job_list = [
  {"sorter_name": "tridesclous", "recording": recording,
            "remove_existing_folder" : True,
            "output_folder" : base_folder / f"output_tridesclous",
            "verbose" : True,
            "freq_min" : 300,
            "freq_max" : 7000.0,
            "detect_threshold" : 5,
            "detect_sign" : -1},
  {"sorter_name": "mountainsort5", "recording": recording,
            "remove_existing_folder" : True,
            "output_folder" : base_folder / f"output_mountainsort5",
            "verbose" : True,
            "scheme": '1',
            "freq_min" : 300,
            "freq_max" : 7000.0,
            "detect_time_radius_msec": 1,
            "detect_threshold": 5},
  {"sorter_name": "spykingcircus2", "recording": recording,
            "remove_existing_folder" : True,
            "output_folder" : base_folder / f"output_spykingcircus2",
            "verbose" : True,
            'general': {'ms_before': 0.5, 'ms_after': 0.5, 'radius_um': 100},
            'filtering': {'freq_min': 300, 'dtype': 'float32'},
            'detection': {'peak_sign': 'neg', 'detect_threshold': 5}},
]


sortings = si.run_sorter_jobs(job_list=job_list, engine='loop')


Local copy of recording


write_binary_recording:   0%|          | 0/1954 [00:00<?, ?it/s]

DataIO <id: 2164724937728> 
  workdir: E:\Work\SpikeSorting\kiehnlab\Treadmill_clustered_txt\Rec_treadmill_6OHDA_xx_-220621_VGlut2-ChR2-64_-220621-095343\output_tridesclous\sorter_output
  sample_rate: 24414.0625
  total_channel: 1
  channel_groups: 0 [ch0]
  nb_segment: 1
  length: 47689728
  durations: 1953.4 s.
catalogue_nested_params
{'chunksize': 24414,
 'clean_cluster': {'apply_auto_merge_cluster': True,
                   'apply_auto_split': True,
                   'apply_trash_low_extremum': True,
                   'apply_trash_not_aligned': True,
                   'apply_trash_small_cluster': True},
 'clean_peaks': {'alien_value_threshold': -1.0, 'mode': 'extremum_amplitude'},
 'cluster_kargs': {'adjacency_radius_um': 0.0,
                   'high_adjacency_radius_um': 0.0,
                   'max_loop': 1000,
                   'min_cluster_size': 20},
 'cluster_method': 'pruningshears',
 'duration': 601.0,
 'extract_waveforms': {'wf_left_long_ms': -2.5,
                  

d:\Applications\anaconda3\envs\eeg\lib\site-packages\neo\rawio\tdtrawio.py:293: UserWarning: Could not identify sev files for channels [1, 1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 8].
  warnings.warn(f"Could not identify sev files for channels {missing_sev_channels}.")


write_binary_recording with n_jobs = 1 and chunk_size = 24414


...

extract waveforms shared_memory multi buffer with n_jobs = 9 and chunk_size = 24414


extract waveforms shared_memory multi buffer:   0%|          | 0/1954 [00:00<?, ?it/s]

find spikes (circus-omp-svd) with n_jobs = 9 and chunk_size = 2441


find spikes (circus-omp-svd):   0%|          | 0/19537 [00:00<?, ?it/s]

We found 241377 spikes
spykingcircus2 run time 122.21s
